# Trino Analytics Homework

## Overview

This notebook demonstrates a complete analytics workflow using Trino for federated queries across PostgreSQL and MySQL databases, with data visualization and long-term storage in Apache Iceberg.

### Architecture

- **Trino**: Distributed SQL query engine for federated queries
- **PostgreSQL**: Source database containing customer and order data
- **MySQL**: Source database containing payment data
- **Apache Iceberg**: Modern table format for analytics with MinIO storage backend
- **Python**: Analytics environment with pandas, matplotlib, and trino libraries

### Workflow Levels

1. **Level 1**: Connections and catalog exploration
2. **Level 2**: Data aggregation and DataFrame creation
3. **Level 3**: Visualization and Iceberg storage

---

## Setup and Imports

Import all required libraries and modules for the analytics workflow.

In [ ]:
# Import required libraries
import sys
import warnings
from datetime import datetime

# Third-party imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Local module imports
sys.path.append('/home/jovyan/work')
from trino_connection import (
    create_trino_connection,
    test_catalog_connectivity,
    execute_sql_query,
    get_catalog_tables,
    test_data_access,
    close_connection
)
from data_aggregation import (
    aggregate_daily_orders,
    aggregate_daily_payments,
    merge_dataframes_with_fillna,
    calculate_payment_coverage,
    create_final_analytics_dataframe,
    get_analytics_summary
)
from visualization import (
    create_time_series_revenue_chart,
    create_payment_coverage_histogram,
    create_combined_analytics_dashboard,
    display_chart_summary
)
from iceberg_storage import (
    save_analytics_to_iceberg,
    query_iceberg_table,
    verify_data_persistence,
    list_iceberg_tables
)

# Configure display settings
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ All modules imported successfully")
print(f"📅 Notebook execution started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---

# Level 1: Connections and Catalog Exploration

In this level, we establish connections to Trino and explore the available catalogs and data sources.

## 1.1 Establish Trino Connection

Create a connection to the Trino coordinator and verify connectivity.

In [ ]:
# Establish connection to Trino
print("🔌 Establishing connection to Trino...")
conn = create_trino_connection()
print("✅ Trino connection established successfully")

## 1.2 Test Catalog Connectivity

Verify that all configured catalogs (PostgreSQL, MySQL, Iceberg) are accessible.

In [ ]:
# Test connectivity to all catalogs
print("🔍 Testing catalog connectivity...")
catalog_status = test_catalog_connectivity(conn)

print("\n📊 Catalog Connectivity Status:")
print("=" * 35)
for catalog, status in catalog_status.items():
    status_icon = "✅" if status else "❌"
    print(f"{status_icon} {catalog:<15}: {'Connected' if status else 'Failed'}")

successful_catalogs = sum(catalog_status.values())
total_catalogs = len(catalog_status)
print(f"\n📈 Summary: {successful_catalogs}/{total_catalogs} catalogs connected successfully")

## 1.3 Explore Available Catalogs

List all available catalogs and their schemas.

In [ ]:
# Show all available catalogs
catalogs_df = execute_sql_query(conn, "SHOW CATALOGS")
print("📚 Available Catalogs:")
display(catalogs_df)

print(f"\nTotal catalogs available: {len(catalogs_df)}")

## 1.4 Explore PostgreSQL Tables

Examine the structure and content of PostgreSQL tables containing order and customer data.

In [ ]:
# Explore PostgreSQL tables
print("🐘 PostgreSQL Tables:")
pg_tables = get_catalog_tables(conn, "postgresql", "public")
display(pg_tables)

# Sample data from key tables
print("\n📋 Sample Customer Data:")
customers_sample = execute_sql_query(conn, """
    SELECT customer_id, customer_name, email, created_at
    FROM postgresql.public.trn_customers
    ORDER BY customer_id
    LIMIT 5
""")
display(customers_sample)

print("\n📦 Sample Orders Data:")
orders_sample = execute_sql_query(conn, """
    SELECT order_id, customer_id, order_ts, total_amount
    FROM postgresql.public.trn_orders
    ORDER BY order_ts DESC
    LIMIT 5
""")
display(orders_sample)

## 1.5 Explore MySQL Tables

Examine the structure and content of MySQL tables containing payment data.

In [ ]:
# Explore MySQL tables
print("🐬 MySQL Tables:")
mysql_tables = get_catalog_tables(conn, "mysql", "demo_db")
display(mysql_tables)

# Sample payment data
print("\n💳 Sample Payment Data:")
payments_sample = execute_sql_query(conn, """
    SELECT payment_id, order_id, amount, paid_at, payment_method
    FROM mysql.demo_db.trn_payments
    ORDER BY paid_at DESC
    LIMIT 5
""")
display(payments_sample)

## 1.6 Test Cross-Catalog Queries

Demonstrate Trino's federated query capabilities by joining data across PostgreSQL and MySQL.

In [ ]:
# Test cross-catalog join
print("🔗 Cross-Catalog Join Example:")
cross_catalog_query = """
SELECT 
    o.order_id,
    o.customer_id,
    o.total_amount as order_amount,
    p.amount as paid_amount,
    p.payment_method,
    o.order_ts,
    p.paid_at,
    CASE 
        WHEN p.amount >= o.total_amount THEN 'Fully Paid'
        WHEN p.amount > 0 THEN 'Partially Paid'
        ELSE 'Unpaid'
    END as payment_status
FROM postgresql.public.trn_orders o
LEFT JOIN mysql.demo_db.trn_payments p ON o.order_id = p.order_id
ORDER BY o.order_ts DESC
LIMIT 10
"""

cross_catalog_df = execute_sql_query(conn, cross_catalog_query)
display(cross_catalog_df)

print(f"\n✅ Successfully executed cross-catalog query returning {len(cross_catalog_df)} rows")

## 1.7 Data Access Verification

Verify that we can access all required tables and get row counts.

In [ ]:
# Test data access across all catalogs
print("📊 Data Access Verification:")
data_access_results = test_data_access(conn)

print("\n📈 Table Row Counts:")
print("=" * 50)
total_rows = 0
for table, count in data_access_results.items():
    if count >= 0:
        print(f"✅ {table:<35}: {count:,} rows")
        total_rows += count
    else:
        print(f"❌ {table:<35}: Access failed")

print(f"\n📊 Total rows across all tables: {total_rows:,}")
print("\n✅ Level 1 completed: Connections and catalog exploration successful!")

---

# Level 2: Data Aggregation and DataFrame Creation

In this level, we perform data aggregation across multiple data sources and create analytical DataFrames.

## 2.1 Daily Orders Aggregation

Aggregate order data by day to calculate daily revenue and order counts.

In [ ]:
# Aggregate daily orders from PostgreSQL
print("📦 Aggregating daily orders data...")
orders_df = aggregate_daily_orders(conn)

print(f"\n📊 Daily Orders Summary:")
print(f"  • Date range: {orders_df['dt'].min()} to {orders_df['dt'].max()}")
print(f"  • Total days: {len(orders_df)}")
print(f"  • Total revenue: ${orders_df['revenue'].sum():,.2f}")
print(f"  • Total orders: {orders_df['orders_cnt'].sum():,}")
print(f"  • Average daily revenue: ${orders_df['revenue'].mean():,.2f}")
print(f"  • Average daily orders: {orders_df['orders_cnt'].mean():.1f}")

print("\n📋 Sample Daily Orders Data:")
display(orders_df.head(10))

## 2.2 Daily Payments Aggregation

Aggregate payment data by day to calculate daily payment amounts and counts.

In [ ]:
# Aggregate daily payments from MySQL
print("💳 Aggregating daily payments data...")
payments_df = aggregate_daily_payments(conn)

print(f"\n📊 Daily Payments Summary:")
print(f"  • Date range: {payments_df['dt'].min()} to {payments_df['dt'].max()}")
print(f"  • Total days: {len(payments_df)}")
print(f"  • Total paid amount: ${payments_df['paid_amount'].sum():,.2f}")
print(f"  • Total payments: {payments_df['payments_cnt'].sum():,}")
print(f"  • Average daily paid amount: ${payments_df['paid_amount'].mean():,.2f}")
print(f"  • Average daily payments: {payments_df['payments_cnt'].mean():.1f}")

print("\n📋 Sample Daily Payments Data:")
display(payments_df.head(10))

## 2.3 DataFrame Merging with Missing Value Handling

Merge orders and payments DataFrames using proper handling of missing values.

In [ ]:
# Merge DataFrames with proper fillna handling
print("🔗 Merging orders and payments DataFrames...")
merged_df = merge_dataframes_with_fillna(orders_df, payments_df, fill_value=0.0)

print(f"\n📊 Merged DataFrame Summary:")
print(f"  • Total rows: {len(merged_df)}")
print(f"  • Date range: {merged_df['dt'].min()} to {merged_df['dt'].max()}")
print(f"  • Columns: {list(merged_df.columns)}")

# Check for missing values after merge
missing_values = merged_df.isnull().sum()
print(f"\n🔍 Missing values after merge:")
for col, count in missing_values.items():
    if count > 0:
        print(f"  • {col}: {count} missing values")
    else:
        print(f"  ✅ {col}: No missing values")

print("\n📋 Sample Merged Data:")
display(merged_df.head(10))

## 2.4 Payment Coverage Calculation

Calculate payment coverage metrics to understand the relationship between orders and payments.

In [ ]:
# Calculate payment coverage metrics
print("📈 Calculating payment coverage metrics...")
coverage_df = calculate_payment_coverage(merged_df)

print(f"\n📊 Payment Coverage Analysis:")
coverage_stats = coverage_df['payment_coverage'].describe()
print(f"  • Average coverage: {coverage_stats['mean']:.2%}")
print(f"  • Median coverage: {coverage_stats['50%']:.2%}")
print(f"  • Maximum coverage: {coverage_stats['max']:.2%}")
print(f"  • Minimum coverage: {coverage_stats['min']:.2%}")
print(f"  • Standard deviation: {coverage_stats['std']:.2%}")

# Coverage distribution analysis
full_coverage_days = (coverage_df['payment_coverage'] >= 1.0).sum()
high_coverage_days = (coverage_df['payment_coverage'] >= 0.8).sum()
low_coverage_days = (coverage_df['payment_coverage'] < 0.5).sum()

print(f"\n📊 Coverage Distribution:")
print(f"  • Days with full coverage (≥100%): {full_coverage_days} ({full_coverage_days/len(coverage_df):.1%})")
print(f"  • Days with high coverage (≥80%): {high_coverage_days} ({high_coverage_days/len(coverage_df):.1%})")
print(f"  • Days with low coverage (<50%): {low_coverage_days} ({low_coverage_days/len(coverage_df):.1%})")

print("\n📋 Sample Coverage Data:")
display(coverage_df.head(10))

## 2.5 Complete Analytics DataFrame Creation

Create the final analytics DataFrame using the complete pipeline.

In [ ]:
# Create final analytics DataFrame using complete pipeline
print("🏗️ Creating final analytics DataFrame...")
final_df = create_final_analytics_dataframe(conn)

print(f"\n📊 Final Analytics DataFrame:")
print(f"  • Shape: {final_df.shape}")
print(f"  • Columns: {list(final_df.columns)}")
print(f"  • Date range: {final_df['dt'].min()} to {final_df['dt'].max()}")

print("\n📋 Final DataFrame Sample:")
display(final_df.head(10))

print("\n📋 Final DataFrame Tail:")
display(final_df.tail(5))

## 2.6 Analytics Summary Generation

Generate comprehensive summary statistics for the analytics data.

In [ ]:
# Generate comprehensive analytics summary
print("📈 Generating analytics summary...")
summary = get_analytics_summary(final_df)

print("\n📊 COMPREHENSIVE ANALYTICS SUMMARY")
print("=" * 50)

print(f"\n📅 Time Period:")
print(f"  • Total days analyzed: {summary['total_days']}")
print(f"  • Date range: {summary['date_range']['start']} to {summary['date_range']['end']}")

print(f"\n💰 Revenue Analysis:")
print(f"  • Total revenue: ${summary['revenue']['total']:,.2f}")
print(f"  • Average daily revenue: ${summary['revenue']['average_daily']:,.2f}")
print(f"  • Maximum daily revenue: ${summary['revenue']['max_daily']:,.2f}")
print(f"  • Minimum daily revenue: ${summary['revenue']['min_daily']:,.2f}")

print(f"\n📦 Orders Analysis:")
print(f"  • Total orders: {summary['orders']['total']:,}")
print(f"  • Average daily orders: {summary['orders']['average_daily']:.1f}")
print(f"  • Maximum daily orders: {summary['orders']['max_daily']:,}")
print(f"  • Minimum daily orders: {summary['orders']['min_daily']:,}")

print(f"\n💳 Payments Analysis:")
print(f"  • Total payments: {summary['payments']['total']:,}")
print(f"  • Total paid amount: ${summary['payments']['total_amount']:,.2f}")
print(f"  • Average daily payment count: {summary['payments']['average_daily_count']:.1f}")
print(f"  • Average daily paid amount: ${summary['payments']['average_daily_amount']:,.2f}")

print(f"\n📈 Payment Coverage Analysis:")
print(f"  • Average coverage: {summary['payment_coverage']['average']:.2%}")
print(f"  • Maximum coverage: {summary['payment_coverage']['max']:.2%}")
print(f"  • Minimum coverage: {summary['payment_coverage']['min']:.2%}")
print(f"  • Days with full coverage: {summary['payment_coverage']['days_with_full_coverage']}")

print("\n✅ Level 2 completed: Data aggregation and DataFrame creation successful!")

---

# Level 3: Visualization and Iceberg Storage

In this level, we create professional visualizations and save the analytics results to Apache Iceberg for long-term storage.

## 3.1 Data Visualization Preparation

Prepare the data for visualization and display summary information.

In [ ]:
# Display chart data summary
print("📊 Preparing data for visualization...")
display_chart_summary(final_df)

# Set visualization style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("\n✅ Data prepared for visualization")

## 3.2 Time-Series Revenue Visualization

Create a professional time-series chart showing daily revenue trends.

In [ ]:
# Create time-series revenue chart
print("📈 Creating time-series revenue chart...")
fig1 = create_time_series_revenue_chart(
    final_df,
    figsize=(14, 8),
    title="Daily Revenue Analysis - Time Series Trend"
)
plt.show()

print("✅ Time-series revenue chart created successfully")

## 3.3 Payment Coverage Distribution

Create a histogram showing the distribution of payment coverage ratios.

In [ ]:
# Create payment coverage histogram
print("📊 Creating payment coverage distribution histogram...")
fig2 = create_payment_coverage_histogram(
    final_df,
    bins=25,
    figsize=(12, 8),
    title="Payment Coverage Distribution Analysis"
)
plt.show()

print("✅ Payment coverage histogram created successfully")

## 3.4 Comprehensive Analytics Dashboard

Create a comprehensive dashboard combining multiple visualizations and summary statistics.

In [ ]:
# Create comprehensive analytics dashboard
print("🎛️ Creating comprehensive analytics dashboard...")
fig3 = create_combined_analytics_dashboard(
    final_df,
    figsize=(18, 12),
    title="Trino Analytics Homework - Comprehensive Dashboard"
)
plt.show()

print("✅ Comprehensive analytics dashboard created successfully")

## 3.5 Iceberg Storage - Schema and Table Creation

Save the analytics results to Apache Iceberg for long-term storage and future analysis.

In [ ]:
# Save analytics data to Iceberg
print("🏔️ Saving analytics data to Apache Iceberg...")
iceberg_results = save_analytics_to_iceberg(
    conn, 
    final_df, 
    table_name="trino_homework_analytics",
    schema_name="homework"
)

print("\n📊 Iceberg Storage Results:")
print("=" * 30)
print(f"✅ Success: {iceberg_results['success']}")
print(f"📋 Table: {iceberg_results['full_table_name']}")
print(f"📊 Rows inserted: {iceberg_results['rows_inserted']:,}")
print(f"🔍 Verification passed: {iceberg_results['verification']['verification_passed']}")
print(f"⏰ Timestamp: {iceberg_results['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}")

if iceberg_results['success']:
    print("\n🎉 Data successfully saved to Iceberg!")
else:
    print("\n⚠️ Iceberg storage completed with issues")

## 3.6 Iceberg Data Verification

Verify that the data was correctly persisted to Iceberg by querying it back.

In [ ]:
# Verify data persistence in Iceberg
print("🔍 Verifying data persistence in Iceberg...")

# Query data back from Iceberg
iceberg_df = query_iceberg_table(
    conn, 
    "trino_homework_analytics", 
    "homework",
    order_by="dt"
)

print(f"\n📊 Iceberg Query Results:")
print(f"  • Rows retrieved: {len(iceberg_df):,}")
print(f"  • Columns: {list(iceberg_df.columns)}")
print(f"  • Date range: {iceberg_df['dt'].min()} to {iceberg_df['dt'].max()}")

# Compare key metrics
print(f"\n🔍 Data Integrity Check:")
original_revenue = final_df['revenue'].sum()
iceberg_revenue = iceberg_df['revenue'].sum()
revenue_match = abs(original_revenue - iceberg_revenue) < 0.01

original_orders = final_df['orders_cnt'].sum()
iceberg_orders = iceberg_df['orders_cnt'].sum()
orders_match = original_orders == iceberg_orders

print(f"  {'✅' if revenue_match else '❌'} Revenue: Original=${original_revenue:,.2f}, Iceberg=${iceberg_revenue:,.2f}")
print(f"  {'✅' if orders_match else '❌'} Orders: Original={original_orders:,}, Iceberg={iceberg_orders:,}")
print(f"  {'✅' if len(final_df) == len(iceberg_df) else '❌'} Row count: Original={len(final_df)}, Iceberg={len(iceberg_df)}")

print("\n📋 Sample Iceberg Data:")
display(iceberg_df.head(5))

if revenue_match and orders_match and len(final_df) == len(iceberg_df):
    print("\n✅ Data integrity verification passed!")
else:
    print("\n⚠️ Data integrity verification found discrepancies")

## 3.7 Iceberg Table Management

Demonstrate Iceberg table management capabilities.

In [ ]:
# List all Iceberg tables in the homework schema
print("📚 Listing Iceberg tables...")
iceberg_tables = list_iceberg_tables(conn, "homework")

if not iceberg_tables.empty:
    print(f"\n📊 Found {len(iceberg_tables)} table(s) in homework schema:")
    for table in iceberg_tables['table_name']:
        print(f"  • {table}")
    display(iceberg_tables)
else:
    print("\nℹ️ No tables found in homework schema")

# Demonstrate analytical query on Iceberg data
print("\n📈 Running analytical query on Iceberg data...")
iceberg_analytics = execute_sql_query(conn, """
    SELECT 
        COUNT(*) as total_days,
        SUM(revenue) as total_revenue,
        AVG(revenue) as avg_daily_revenue,
        SUM(orders_cnt) as total_orders,
        AVG(payment_coverage) as avg_coverage,
        COUNT(CASE WHEN payment_coverage >= 1.0 THEN 1 END) as full_coverage_days
    FROM iceberg.homework.trino_homework_analytics
""")

print("\n📊 Iceberg Analytics Summary:")
display(iceberg_analytics)

print("\n✅ Iceberg table management demonstration completed")

---

# Final Summary and Cleanup

Summarize the complete workflow and clean up resources.

## Workflow Summary

This notebook successfully demonstrated a complete analytics workflow using Trino for federated queries.

In [ ]:
# Final workflow summary
print("🎯 TRINO ANALYTICS HOMEWORK - FINAL SUMMARY")
print("=" * 60)

print("\n✅ LEVEL 1 - Connections and Catalog Exploration:")
print("  • Established connection to Trino coordinator")
print("  • Verified connectivity to PostgreSQL, MySQL, and Iceberg catalogs")
print("  • Explored table structures and sample data")
print("  • Demonstrated cross-catalog federated queries")

print("\n✅ LEVEL 2 - Data Aggregation and DataFrame Creation:")
print("  • Aggregated daily orders data from PostgreSQL")
print("  • Aggregated daily payments data from MySQL")
print("  • Merged DataFrames with proper missing value handling")
print("  • Calculated payment coverage metrics")
print("  • Generated comprehensive analytics summary")

print("\n✅ LEVEL 3 - Visualization and Iceberg Storage:")
print("  • Created professional time-series revenue charts")
print("  • Generated payment coverage distribution histograms")
print("  • Built comprehensive analytics dashboard")
print("  • Saved results to Apache Iceberg for long-term storage")
print("  • Verified data persistence and integrity")

print(f"\n📊 KEY METRICS:")
print(f"  • Total days analyzed: {len(final_df)}")
print(f"  • Total revenue: ${final_df['revenue'].sum():,.2f}")
print(f"  • Total orders: {final_df['orders_cnt'].sum():,}")
print(f"  • Average payment coverage: {final_df['payment_coverage'].mean():.2%}")
print(f"  • Data successfully persisted to Iceberg: {'Yes' if iceberg_results['success'] else 'No'}")

print(f"\n⏰ Execution completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n🎉 HOMEWORK COMPLETED SUCCESSFULLY!")

## Resource Cleanup

Clean up connections and resources.

In [ ]:
# Close Trino connection
print("🧹 Cleaning up resources...")
close_connection(conn)
print("✅ Trino connection closed successfully")

# Clear large variables to free memory
del final_df, orders_df, payments_df, merged_df, coverage_df, iceberg_df
print("✅ Memory cleanup completed")

print("\n🏁 All resources cleaned up successfully!")
print("\n" + "=" * 60)
print("Thank you for using Trino Analytics Homework!")
print("=" * 60)